# Monthly GDP Tables: GVA

Quite a dense spreadsheet from the UK Office for National Statistics. Has a non obvious pattern of sporadically interleaving header rows (weight, CDID's) mixed in with the rows of observational data.

_Note: I particuarly like this example, it _looks_ like it should be nightmare to process (and is with most tooling), but tidychef nails this trivially. This tends to be one of penny drops moments for people._

## Tutorial Structure

With these example tutorials I'm going to comment heavily and cover nuances in a follow up section (with liberal targetted previews as needed) as it's the easiest way to grapple with new ideas. It may also be worth opening up these notebooks yourself (they're in `./jupyterbook` in the [tidychef](https://github.com/mikeAdamss/tidychef) github repo) so you can run, alter and generally have a play about with this yourself as part of your learning.

We'll cover:

- source data
- requirements, what we're aiming to do here
- show the full script (all logic commented)
- output the selection preview
- nuances (where applicable)
- view the output

This sequencing is necessary as the output for some of the example is **really** long so that necessitates it coming last. If you're viewing this via a jupyter book (i.e on the site) you can navigate between the above sections via your right hand menu.

_Note - these tutorial scripts might seem verbose due all the comments but that's ok (this is a tutorial after all). If you take them out you end up with a fairly succinct and human readable encapsulation of what would otherwise (with existing tools) be a rather convoluted and fragile set of instructions to express._

In virtually all cases I'll make heavy use of `preview` and `bounded` to only look at relevant parts of what can be quite large datasets. Downloads links are provided for the source data.

## Source

For this example we're extracting the table "GVA" as shown below (note - preview cropped to row 16 for reasons of practicality):

In [1]:
from tidychef import acquire, preview
from tidychef.selection import XlsSelectable

table: XlsSelectable = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/monthlygdptablesapril2023.xls", tables="GVA")
preview(table, bounded="A1:Z16")

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
1,"GVA - Gross Value Added [note 1] & Sections A-T [note 3], [note 4], [note 5]",,,,,,,,,,,,,,,,,,,,,,,,,
2,This worksheet contains one table. Some cells refer to notes which can be found on the notes worksheet.,,,,,,,,,,,,,,,,,,,,,,,,,
3,Link to return to cover page,,,,,,,,,,,,,,,,,,,,,,,,,
4,"Source: GDP monthly estimate, Office for National Statistics",,,,,,,,,,,,,,,,,,,,,,,,,
5,Time Period,Category,Total GVA at basic prices (A - T),"Agriculture, forestry and fishing (A)",Total production industries (B - E),Mining and Quarrying (B),Manufacturing (C),"Electricity, gas, steam and air (D)","Water supply, sewerage etc (E)",Construction (F) [note 6],Total service industries (G-T),Wholesale and retail: repair of motor vehicles and motorcycles (G),Transport and storage (H),Accommodation and food service activites (I),Information and communication (J),Financial and insurance activities (K),Real estate activites (L),"Professional, scientific and technical activities (M)",Administrative and support service activities (N),Public administration and defence (O),Education (P),Human health and social work activities (Q),"Arts, entertainment and recreation (R)",Other service activities (S),"Activities of households as employers, undifferentiated goods and services (T)",
6,2019.0,Weight,1000.0,7.0,135.0,11.0,97.0,15.0,12.0,62.0,796.0,104.0,40.0,30.0,63.0,82.0,132.0,73.0,51.0,49.0,60.0,77.0,15.0,17.0,2.0,
7,[Not applicable],CDID,YBFR,L2KL,L2KQ,L2KR,L2KX,L2MW,L2N2,L2N8,L2NC,L2NE,L2NI,L2NQ,L2NT,L2O6,L2OC,L2OI,L2OX,L2P8,L2PA,L2PC,L2PJ,L2PP,L2PT,
8,2018.0,Annual Chained Volume Index [note 2],98.3,85.6,97.7,97.9,98.9,87.7,100.3,99.0,98.5,100.4,98.2,96.6,91.6,102.6,98.4,100.9,99.6,95.0,94.0,101.6,98.3,101.8,94.1,
9,2019.0,Annual Chained Volume Index [note 2],100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,
10,2020.0,Annual Chained Volume Index [note 2],89.4,96.5,101.3,96.8,100.1,109.1,104.8,86.0,87.6,81.6,70.3,59.9,98.6,100.3,100.0,92.5,89.0,95.2,82.5,69.1,71.9,86.3,78.0,


From an xls source which can be [downloaded here](https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/monthlygdptablesapril2023.xls).

One interesting thing to note here is the producer has intermingled CDID identifiers in with the primary observations values (see lines 7 and 13 above) - we need to be sure to account for this.

# Requirements

- We'll take cell C5 and cells directly right as the column "Identifier"
- We'll take the weight row as weighting for all observations below it
- We'll take cell C8 and cells directly downwards as the column "Category"
- We'll take all cells to the right of a column B value of "CDID" as the column "CDID"
- We'll take cell A8 and cells directly downwards as "Time Period"
- We're going to ignore the "Weight" values for this example.
- We're also going to ignore the bracketed text and just remove it for our purposes here.
- We'll take the observations are the principle table values minus the CDID headings that are intermingled. We'll use a column name of "Value" for them.

In [6]:
from tidychef import acquire, preview
from tidychef.direction import up, down, left, right
from tidychef.output import TidyData, Column
from tidychef.selection import XlsSelectable

table: XlsSelectable = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/monthlygdptablesapril2023.xls", tables="GVA")

# So given its a solid table we're going to create an "anchor", a cell in the top left corner
# we can make our other selections relative to.
anchor = table.excel_ref("B").cell_containing_string("Category").label_as("Anchor")

# Identifiers are just headers to the right of the anchor
identifier = anchor.fill(right).is_not_blank().label_as("Identifiers")

# Weighting is the row below the anchor
weighting = anchor.shift(down).fill(right).is_not_blank().label_as("Weighting")

# We're gonna create a selection we'll call data_block
# imagine you just hold click C8 and drag down right until you run out of spreadhseet - same thing
data_block = anchor.shift(down(2)).shift(right).expand_to_box()

# CDID's are non numeric values in the data_block
cdid = data_block.is_not_numeric().label_as("CDID")

# observations are numeric values in the data block
observations = data_block.is_numeric().label_as("Observations")

# our categories are anything below the anchor thats not to the left of weighting or cdid headers
category = anchor.fill(down).is_not_blank().label_as("Category") - weighting.fill(left) - cdid.fill(left)

# time period is whatevers left of categories
time_period = category.shift(left).label_as("Time Period")

# Create a bounded preview inline but also write the full preview to path
preview(anchor, identifier, weighting, cdid, category, time_period, observations, bounded="A4:Z16")
preview(anchor, identifier, weighting, cdid, category, time_period, observations, path="monthly-gdp-gva-table.html")

# Derive the unit of measure
# I'll touch on this in nunaces
def derive_unit_of_measure(row) -> str:
    category_cell = row["Category"].lower()
    if "percentage" in category_cell:
        return "%"
    elif "index" in category_cell:
        return "Index"
    raise ValueError(f"Cannot derive unit of measure from {category_cell}")
                           
tidy_data = TidyData(
    observations,
    Column(identifier.attach_directly(down)),
    Column(weighting.attach_directly(down)),
    Column(cdid.attach_directly(down)),
    Column(category.attach_directly(right)),
    Column(time_period.attach_directly(right), apply=lambda x: x.rstrip(".0")), # dont have years with .0 on the end
    Column.horizontal_condition("Unit", derive_unit_of_measure)
)

tidy_data.to_csv("monthly-gdp-gva-table.csv")

## Nuances

This is pretty straight forward visual wrangling.The only exception is the `horizontal_condition` (which we'll cover properly in the intermediate level documentation - we're jumping ahead a bit here).

It should however still make an amount of sense to someone familiar with python, the idea is basically `derived` not extracted columns. In this instance:

```python
# This is the function that matters!
def derive_unit_of_measure(row) -> str:
    category_cell = row["Category"].lower()
    if "percentage" in category_cell:
        return "%"
    elif "index" in category_cell:
        return "Index"
    raise ValueError(f"Cannot derive unit of measure from {category_cell}")
                           
tidy_data = TidyData(
    observations,
    Column(identifier.attach_directly(down)),
    Column(weighting.attach_directly(down)),
    Column(cdid.attach_directly(down)),
    Column(category.attach_directly(right)),
    Column(time_period.attach_directly(right)),
    Column.horizontal_condition("Unit", derive_unit_of_measure)   #  <---- This is where its called
)
```

So what we're doing is:

- we want a column called "Unit"
- if the string "percentage" is in the Category column for a row - populate with %
- if the string "index" is in the Category column for a row - populate with Index
- throw an error of anything not covered by the above (example of good practice/future proofing).

Note - if the above isn't obvious to you don't worry, it will be by the time we get to the intermedaite section and into the `horiziontal_condition` documentation.

# Outputs

The full preview can be [viewed here](./monthly-gdp-gva-table.html):

The tidy data can be [downloaded here](./monthly-gdp-gva-table.html) and a full inline preview of the tidydata generated is shown below for those people who'd prefer to scroll.

In [7]:
print(tidy_data)

Observations,Identifiers,Weighting,CDID,Category,Time Period,Unit
98.3,Total GVA at basic prices (A - T),1000.0,YBFR,Annual Chained Volume Index [note 2],2018,Index
100.0,Total GVA at basic prices (A - T),1000.0,YBFR,Annual Chained Volume Index [note 2],2019,Index
89.4,Total GVA at basic prices (A - T),1000.0,YBFR,Annual Chained Volume Index [note 2],202,Index
96.1,Total GVA at basic prices (A - T),1000.0,YBFR,Annual Chained Volume Index [note 2],2021,Index
100.2,Total GVA at basic prices (A - T),1000.0,YBFR,Annual Chained Volume Index [note 2],2022,Index
95.1,Total GVA at basic prices (A - T),1000.0,ECY2,Monthly Chained Volume Index [note 2],2021 Apr,Index
96.3,Total GVA at basic prices (A - T),1000.0,ECY2,Monthly Chained Volume Index [note 2],2021 May,Index
97.3,Total GVA at basic prices (A - T),1000.0,ECY2,Monthly Chained Volume Index [note 2],2021 Jun,Index
97.2,Total GVA at basic prices (A - T),1000.0,ECY2,Monthly Chained Volume Index [note 2],2021 Jul,Index
98.0,Total GVA at basic prices (A - T),1000.0,ECY2,Monthly Chained Volume Index [note 2],2021 Aug,Index
